In [1]:
import sys, os
import torch, wandb
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))

In [2]:
from configs import example_unet_config as config
from models.example_unet import SimpleUNet
from datasets.depth_dataset import DepthDataset
from utils.train_utils import train_model

/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# Set a fixed random seed for reproducibility
torch.manual_seed(config.random_seed)

train_full_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'train/train'),
    list_file=os.path.join(config.dataset_path, 'train_list.txt'), 
    transform=config.transform_train,
    target_transform=config.target_transform,
    has_gt=True)
    
    # Create test dataset without ground truth
test_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'test/test'),
    list_file=os.path.join(config.dataset_path, 'test_list.txt'),
    transform=config.transform_val,
    has_gt=False)  # Test set has no ground truth
    
# Split training dataset into train and validation
total_size = len(train_full_dataset)
train_size = int((1-config.val_part) * total_size)  
val_size = total_size - train_size    
    
train_dataset, val_dataset = torch.utils.data.random_split(
    train_full_dataset, [train_size, val_size]
)
#val_dataset.transform = config.transform_val # I dont think we need to use augmentations for validation

# Create data loaders with memory optimizations
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.train_bs, 
    shuffle=True, 
    num_workers=config.num_workers, 
    pin_memory=True,
    drop_last=True,
    persistent_workers=True
)
    
    
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)
    
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 20375, Validation size: 3596, Test size: 650


In [4]:
model = config.model()
# #model = nn.DataParallel(model)


optimizer = config.optimizer(model.parameters())
print(f"Using device: {config.device}")

Using device: cuda:0


In [5]:
checkpoint = torch.load('/home/v.lomtev/CIL/CIL_2025/data/provided-baseline/best_model_37.pt',
                        map_location=config.device)
model.load_state_dict(checkpoint)

/tmp/ipykernel_2130498/3928104919.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/home/v.lomtev/CIL/CIL_2025/data/provided-baseline/best_model_

<All keys matched successfully>

In [5]:
print("Starting training...")
exp_name = "provided-baseline"
with wandb.init(project="CIL",
                save_code=True,
                notes=config.WANDB_NOTES):
    model = train_model(model, train_loader, val_loader,
                        config.loss, optimizer, config.epochs, config.device,
                       exp_path=os.path.join(config.dataset_path, exp_name))

Starting training...


wandb: Currently logged in as: noloo to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/40


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:31<00:00,  7.23it/s]


Train Loss: 2.4494, Validation Loss: 2.2113
New best model saved at epoch 1 with validation loss: 2.2113
Epoch 2/40


...

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:30<00:00,  7.33it/s]


Train Loss: 1.9000, Validation Loss: 1.7868
New best model saved at epoch 4 with validation loss: 1.7868
Epoch 5/40


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:30<00:00,  7.31it/s]


Train Loss: 1.8441, Validation Loss: 1.7738
New best model saved at epoch 5 with validation loss: 1.7738
Epoch 6/40


Training:  71%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                            | 908/1273 [04:32<01:49,  3.34it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
import utils.train_utils as tu

In [7]:
from utils.train_utils import evaluate_model


tu.evaluate_model(model, val_loader, config.device,
                  exp_path=os.path.join(config.dataset_path, exp_name))

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:44<00:00,  5.11it/s]


{'MAE': 0.9072383081278736,
 'RMSE': 1.2238863947417307,
 'siRMSE': 0.30741940761550646,
 'REL': 0.3799060202747585,
 'Delta1': 0.4349994001364509,
 'Delta2': 0.7353936559662013,
 'Delta3': 0.8927475576018216}

In [10]:

tu.generate_test_predictions(model, test_loader, config.device,
                             exp_path=os.path.join(config.dataset_path, exp_name))

Generating Test Predictions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:05<00:00,  7.18it/s]


In [ ]:
exp_name = "provided-baseline"
tu.visualize_test_predictions(model, test_loader, config.device,
                              exp_path=os.path.join(config.dataset_path, exp_name))

Visualizing:  10%|███████████████                                                                                                                                            | 4/41 [01:23<12:47, 20.75s/it]